# $U(1)$ Gauge Model using L2HMC in graph mode

--------------------------------------------------------------------
### TODO:
* [ ] Look at performance on Cooley (longer training runs).
* [ ] Fit observables to Eq. \ref{eq:therm_time} to determine the thermalization time $\tau$.

\begin{equation} 
f(t) \equiv A \exp^{-t / \tau}+ \,\, B
\label{eq:therm_time}
\end{equation}
* [ ] Look at defining a distance metric as the difference in topological charge between two samples and see what effect adding this as an additional term to the loss function has on the models' ability to tunnel between topological sectors.

--------------------------------------------------------------------

In [1]:
import os
import sys
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.special import i0, i1

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import utils.gauge_model_helpers as helpers
from utils.data_utils import (
    calc_avg_vals_errors, block_resampling, jackknife_err
)
from gauge_model import (
    GaugeModel, check_else_make_dir, save_params_to_pkl_file
)

tfe = tf.contrib.eager
%autoreload 2
%matplotlib notebook

#from gauge_model import GaugeModel
#%reload_ext gauge_model.GaugeModel
#from gauge_model import GaugeModel
#
#from dynamics.gauge_dynamics import GaugeDynamics
#%reload_ext dynamics.gauge_dynamics
#from dynamics.gauge_dynamics import GaugeDynamics

In [2]:
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    result /= result[result.argmax()]
    return result[result.size//2:]

## Define parameters

In [10]:
params = {
#--------------------- Lattice parameters ----------------------------
    'time_size': 8,
    'space_size': 8,
    'link_type': 'U1',
    'dim': 2,
    'num_samples': 4,
    'rand': False,
#--------------------- Leapfrog parameters ---------------------------
    'num_steps': 5,
    'eps': 0.1,
    'loss_scale': 1.,
    'loss_eps': 1e-4,
#--------------------- Learning rate parameters ----------------------
    'learning_rate_init': 1e-3,
    'learning_rate_decay_steps': 500,
    'learning_rate_decay_rate': 0.96,
#--------------------- Annealing rate parameters ---------------------
    'annealing': True,
    'annealing_steps': 500,
    'annealing_factor': 0.99995,
    #'beta': 3.,
    'beta_init': 3.,
    'beta_final': 8.,
#--------------------- Training parameters ---------------------------
    'train_steps': 20000,
    'save_steps': 1000,
    'logging_steps': 50,
    'training_samples_steps': 500,
    'training_samples_length': 100,
#--------------------- Model parameters ------------------------------
    'conv_net': True,
    'hmc': False,
    'eps_trainable': True,
    'metric': 'l2',
    'aux': True,
    'clip_grads': False,
    'clip_value': 10.,
}

## Create model

In [11]:
os.environ["KMP_BLOCKTIME"] = str(0)
os.environ["KMP_SETTINGS"] = str(1)
os.environ["KMP_AFFINITY"] = "granularity=fine,verbose,compact,1,0"
#os.environ["OMP_NUM_THREADS"] = str(2)

In [12]:
config = tf.ConfigProto()
tf.reset_default_graph()

In [6]:
model = GaugeModel(params=params, 
                   config=config,
                   sess=None,
                   log_dir=None,
                   restore=False)

save_params_to_pkl_file(params, model.info_dir)

Creating directory for new run: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_34/
Creating directory: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_34/samples_history.
Creating directory: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_34/samples_history/training.
Creating directory: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_34/train_samples.
time_size: 8
space_size: 8
link_type: U1
dim: 2
num_samples: 4
rand: False
num_steps: 5
eps: 0.1
loss_scale: 1.0
loss_eps: 0.0001
learning_rate_init: 0.001
learning_rate_decay_steps: 500
learning_rate_decay_rate: 0.96
annealing: True
annealing_steps: 200
annealing_factor: 0.9998
beta_init: 3.0
beta_final: 8.0
train_steps: 5000
save_steps: 1000
logging_steps: 50
training_samples_steps: 500
training_samples_length: 100
conv_net: True
hmc: False
eps_trainable: True
metric: l2
aux: True
clip_grads: False
clip_value: 10.0
--------------------------------------------------------------------------------

Building graph... (started at: Thu Jan 31 19

In [7]:
model.train(model.train_steps, kill_sess=False)

----------------------------------------------------------------------------------------------------
     STEP           LOSS       NORM. TIME     ACCEPT %        EPS           BETA           LR      
----------------------------------------------------------------------------------------------------
     0/5000        -4229        0.0723        0.9183          0.1            3           0.001    
     5/5000        -1475       5.627e-05     0.008389       0.09549        3.003         0.001    
    10/5000        -1186       4.793e-05       0.25         0.09276        3.006         0.001    
    15/5000        -1590       4.791e-05       0.75         0.09015        3.009         0.001    
    20/5000       -4.205       5.127e-05      0.00323       0.08836        3.012         0.001    
    25/5000        -2664       6.715e-05      0.7514        0.08721        3.015         0.001    
    30/5000       -644.6       4.672e-05      0.5105        0.08647        3.018         0.001    
    3

   400/5000        -2802       4.678e-05      0.9956        0.01004        3.25          0.001    
   405/5000        -2660       4.973e-05      0.9947       0.009527        3.253         0.001    
   410/5000        -2521       5.26e-05       0.9999       0.009086        3.256         0.001    
   415/5000        -2653       5.587e-05      0.9954       0.008618        3.26          0.001    
   420/5000        -2610       4.229e-05      0.9986       0.008197        3.263         0.001    
   425/5000        -2677       4.683e-05      0.9957       0.007964        3.266         0.001    
   430/5000        -2745       4.926e-05      0.9973       0.007827        3.269         0.001    
   435/5000        -2478       5.355e-05      0.9942       0.007663        3.273         0.001    
   440/5000        -2389       4.576e-05      0.9992       0.007456        3.276         0.001    
   445/5000        -2554       4.369e-05      0.9954       0.007282        3.279         0.001    
   450/500

   690/5000        -2446       5.025e-05      0.9989        0.00733        3.444        0.00096   
   695/5000        -2507       4.846e-05      0.9966       0.007431        3.447        0.00096   
   700/5000        -2906       4.86e-05       0.9967       0.007614        3.451        0.00096   
   705/5000        -2407       5.406e-05         1         0.007835        3.454        0.00096   
   710/5000        -2511       5.641e-05      0.9972       0.007906        3.458        0.00096   
   715/5000        -2562       4.601e-05       0.999       0.007982        3.461        0.00096   
   720/5000        -2758       6.086e-05      0.9979       0.007888        3.465        0.00096   
   725/5000        -2701       5.398e-05       0.999       0.007656        3.468        0.00096   
   730/5000        -2593       5.301e-05      0.9968       0.007476        3.472        0.00096   
   735/5000        -2494       6.424e-05      0.9983       0.007381        3.475        0.00096   
   740/500

  1000/5000        -2538       4.458e-05      0.9979       0.007577        3.664       0.0009216  
  1005/5000        -2492       4.922e-05      0.9929       0.007755        3.668       0.0009216  
  1010/5000        -2630       5.901e-05      0.9971       0.008065        3.672       0.0009216  
  1015/5000        -2600       4.631e-05      0.9978        0.00846        3.675       0.0009216  
  1020/5000        -2613       4.971e-05      0.9987       0.008788        3.679       0.0009216  
  1025/5000        -2506       5.006e-05      0.9952       0.008902        3.683       0.0009216  
  1030/5000        -2399       4.488e-05      0.9982       0.008908        3.686       0.0009216  
  1035/5000        -2242       4.974e-05      0.9996       0.008881        3.69        0.0009216  
  1040/5000        -2509       3.673e-05         1          0.0089         3.694       0.0009216  
  1045/5000        -2472       3.609e-05      0.9998       0.008844        3.697       0.0009216  
  1050/500

  1415/5000        -2632       4.51e-05       0.9981        0.00793        3.981       0.0009216  
  1420/5000        -2555       4.023e-05       0.999       0.007706        3.985       0.0009216  
  1425/5000        -2641       3.946e-05      0.9968       0.007662        3.989       0.0009216  
  1430/5000        -2541       3.942e-05      0.9998       0.007728        3.993       0.0009216  
  1435/5000        -2568       3.676e-05         1         0.007634        3.997       0.0009216  
  1440/5000        -2553       3.773e-05      0.9985       0.007566        4.001       0.0009216  
  1445/5000        -2567       3.398e-05      0.9971       0.007805        4.005       0.0009216  
  1450/5000        -2696       4.887e-05       0.997       0.008217        4.009       0.0009216  
  1455/5000        -2533       4.305e-05      0.9987       0.008576        4.013       0.0009216  
  1460/5000        -2612       3.96e-05       0.9984       0.008839        4.017       0.0009216  
  1465/500

...

  3710/5000        -2231       5.535e-05      0.9977       0.008935        6.301       0.0007514  
  3715/5000        -2384       6.016e-05      0.9984       0.008207        6.307       0.0007514  
  3720/5000        -2479       0.0001141      0.9928       0.007577        6.313       0.0007514  
  3725/5000        -2597       6.448e-05      0.9975       0.006953        6.32        0.0007514  
  3730/5000        -2606       5.554e-05      0.9989       0.006751        6.326       0.0007514  
  3735/5000        -2511       6.416e-05      0.9983       0.007349        6.332       0.0007514  
  3740/5000        -2459       6.222e-05      0.9972       0.007955        6.339       0.0007514  
  3745/5000        -2511       5.381e-05      0.9962       0.007912        6.345       0.0007514  
  3750/5000        -2452       5.303e-05      0.9983       0.007404        6.351       0.0007514  
  3755/5000        -2490       6.97e-05        0.996       0.007309        6.358       0.0007514  
  3760/500

  4005/5000        -2539       6.942e-05       0.999       0.007348        6.684       0.0007214  
  4010/5000        -2477       5.928e-05      0.9985       0.008548        6.691       0.0007214  
  4015/5000        -2549       6.042e-05      0.9984        0.00916        6.697       0.0007214  
  4020/5000        -2653       7.79e-05       0.9992        0.00903        6.704       0.0007214  
  4025/5000        -2754       9.849e-05      0.9946       0.008386        6.711       0.0007214  
  4030/5000        -2567       7.281e-05      0.9969        0.00667        6.717       0.0007214  
  4035/5000        -2446       5.244e-05      0.9994       0.005842        6.724       0.0007214  
  4040/5000        -2512       7.029e-05      0.9957       0.007034        6.731       0.0007214  
  4045/5000        -2437       5.733e-05      0.9988       0.008273        6.738       0.0007214  
  4050/5000        -2285       8.077e-05      0.9975       0.008776        6.744       0.0007214  
  4055/500

  4420/5000        -2563       9.139e-05      0.9973       0.009761        7.262       0.0007214  
  4425/5000        -2581       7.889e-05      0.9967       0.009487        7.27        0.0007214  
  4430/5000        -2532       8.744e-05         1         0.009206        7.277       0.0007214  
  4435/5000        -2623       8.43e-05       0.9969       0.008926        7.284       0.0007214  
  4440/5000        -2583       5.399e-05      0.9982       0.008199        7.291       0.0007214  
  4445/5000        -2469       5.971e-05      0.9976       0.007445        7.299       0.0007214  
  4450/5000        -2528        6.2e-05          1         0.007287        7.306       0.0007214  
  4455/5000        -2558       8.721e-05      0.9977       0.007873        7.313       0.0007214  
  4460/5000        -2480       6.082e-05      0.9961       0.008238        7.321       0.0007214  
  4465/5000        -2717       6.02e-05       0.9941        0.00854        7.328       0.0007214  
  4470/500

  4710/5000        -2592       9.564e-05         1         0.008783        7.696       0.0006925  
  4715/5000        -2716       5.171e-05      0.9989       0.008727        7.704       0.0006925  
  4720/5000        -2397       0.000109       0.9983        0.00821        7.711       0.0006925  
  4725/5000        -2732       5.849e-05      0.9981       0.007757        7.719       0.0006925  
  4730/5000        -2293       6.19e-05        0.998       0.006655        7.727       0.0006925  
  4735/5000        -2447       6.126e-05      0.9957       0.007503        7.735       0.0006925  
  4740/5000        -2330       5.618e-05      0.9994       0.008425        7.742       0.0006925  
  4745/5000        -2554       5.265e-05      0.9999        0.00918        7.75        0.0006925  
  4750/5000        -2568       6.364e-05      0.9947       0.009108        7.758       0.0006925  
  4755/5000        -2620       5.714e-05         1         0.008317        7.766       0.0006925  
  4760/500

Saving checkpoint to: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_34/model.ckpt



In [8]:
run_steps = [100, 200, 500, 1000, 5000]#, 10000]

In [9]:
for steps in run_steps:
    _ = model.run(steps)

Running (trained) L2HMC sampler for 100 steps...
step:      0/100    accept prob (avg):     1     step size: 0.008271 	 time/step: 0.1364

accept prob:  [1. 1. 1. 1.]


step:     10/100    accept prob (avg):  0.9973   step size: 0.008271 	 time/step: 0.02346

accept prob:  [0.99299806 0.9963957  1.         1.        ]


step:     20/100    accept prob (avg):  0.9936   step size: 0.008271 	 time/step: 0.0196

accept prob:  [0.992496   1.         0.99343175 0.98848236]


step:     30/100    accept prob (avg):  0.9967   step size: 0.008271 	 time/step: 0.02868

accept prob:  [0.99425626 0.99862766 1.         0.99399614]


step:     40/100    accept prob (avg):  0.9936   step size: 0.008271 	 time/step: 0.03636

accept prob:  [0.9944547  0.99726814 0.9903794  0.9922742 ]


step:     50/100    accept prob (avg):  0.9967   step size: 0.008271 	 time/step: 0.01947

accept prob:  [0.99327385 1.         0.99357355 1.        ]


step:     60/100    accept prob (avg):  0.9983   step size: 0.00827

step:    190/500    accept prob (avg):  0.9984   step size: 0.008271 	 time/step: 0.02161

accept prob:  [0.99591184 0.9984947  0.9991338  1.        ]


step:    200/500    accept prob (avg):   0.999   step size: 0.008271 	 time/step: 0.01797

accept prob:  [0.99970764 0.99611884 1.         1.        ]


step:    210/500    accept prob (avg):  0.9981   step size: 0.008271 	 time/step: 0.02106

accept prob:  [0.9990651  1.         0.99788785 0.9955529 ]


step:    220/500    accept prob (avg):  0.9983   step size: 0.008271 	 time/step: 0.02292

accept prob:  [0.9962669 1.        0.9968672 1.       ]


step:    230/500    accept prob (avg):  0.9978   step size: 0.008271 	 time/step: 0.02779

accept prob:  [1.        0.9964065 0.99482   1.       ]


step:    240/500    accept prob (avg):  0.9993   step size: 0.008271 	 time/step: 0.02201

accept prob:  [1.        0.9998613 0.9972779 1.       ]


step:    250/500    accept prob (avg):  0.9981   step size: 0.008271 	 time/step: 0.0257

acce

step:    220/1000   accept prob (avg):  0.9978   step size: 0.008271 	 time/step: 0.02031

accept prob:  [0.99982595 0.9957371  1.         0.9957201 ]


step:    230/1000   accept prob (avg):  0.9994   step size: 0.008271 	 time/step: 0.02211

accept prob:  [1.        1.        0.9997119 0.9978814]


step:    240/1000   accept prob (avg):  0.9992   step size: 0.008271 	 time/step: 0.02111

accept prob:  [0.99958   0.9971514 1.        1.       ]


step:    250/1000   accept prob (avg):   0.999   step size: 0.008271 	 time/step: 0.02498

accept prob:  [0.99603593 1.         1.         1.        ]


step:    260/1000   accept prob (avg):  0.9928   step size: 0.008271 	 time/step: 0.021 

accept prob:  [0.98883003 0.9925333  0.99878985 0.9909021 ]


step:    270/1000   accept prob (avg):  0.9973   step size: 0.008271 	 time/step: 0.02391

accept prob:  [1.         0.9929133  0.99788874 0.99852294]


step:    280/1000   accept prob (avg):     1     step size: 0.008271 	 time/step: 0.01864



step:    770/1000   accept prob (avg):   0.997   step size: 0.008271 	 time/step: 0.02869

accept prob:  [0.98783904 1.         1.         1.        ]


step:    780/1000   accept prob (avg):  0.9951   step size: 0.008271 	 time/step: 0.01824

accept prob:  [1.         0.9939123  0.9931304  0.99331343]


step:    790/1000   accept prob (avg):  0.9978   step size: 0.008271 	 time/step: 0.01876

accept prob:  [0.9990042  0.9961566  0.99589205 1.        ]


step:    800/1000   accept prob (avg):  0.9978   step size: 0.008271 	 time/step: 0.01927

accept prob:  [0.9983317 0.9949365 0.9980961 1.       ]


step:    810/1000   accept prob (avg):  0.9976   step size: 0.008271 	 time/step: 0.01966

accept prob:  [0.99138683 0.99969316 1.         0.9991859 ]


step:    820/1000   accept prob (avg):  0.9973   step size: 0.008271 	 time/step: 0.01973

accept prob:  [0.9964819 1.        0.994838  0.9978094]


step:    830/1000   accept prob (avg):  0.9963   step size: 0.008271 	 time/step: 0.0193



step:    310/5000   accept prob (avg):   0.997   step size: 0.008271 	 time/step: 0.01704

accept prob:  [0.9984047 1.        0.9937065 0.9959569]


step:    320/5000   accept prob (avg):  0.9998   step size: 0.008271 	 time/step: 0.01924

accept prob:  [0.99978435 1.         1.         0.99936223]


step:    330/5000   accept prob (avg):  0.9988   step size: 0.008271 	 time/step: 0.02169

accept prob:  [1.        0.9951429 1.        1.       ]


step:    340/5000   accept prob (avg):  0.9968   step size: 0.008271 	 time/step: 0.01929

accept prob:  [0.99602675 1.         0.9957983  0.9954211 ]


step:    350/5000   accept prob (avg):  0.9976   step size: 0.008271 	 time/step: 0.01951

accept prob:  [0.9953438  1.         0.99710923 0.99781495]


step:    360/5000   accept prob (avg):  0.9985   step size: 0.008271 	 time/step: 0.01963

accept prob:  [1.        0.9940171 1.        1.       ]


step:    370/5000   accept prob (avg):  0.9988   step size: 0.008271 	 time/step: 0.02215

acc

...

step:   2510/5000   accept prob (avg):  0.9987   step size: 0.008271 	 time/step: 0.02166

accept prob:  [0.9961985  0.99875355 1.         1.        ]


step:   2520/5000   accept prob (avg):  0.9992   step size: 0.008271 	 time/step: 0.02453

accept prob:  [0.99666244 1.         1.         1.        ]


step:   2530/5000   accept prob (avg):  0.9994   step size: 0.008271 	 time/step: 0.0227

accept prob:  [1.        1.        0.9977234 1.       ]


step:   2540/5000   accept prob (avg):  0.9991   step size: 0.008271 	 time/step: 0.02439

accept prob:  [1.         0.99652207 1.         1.        ]


step:   2550/5000   accept prob (avg):  0.9975   step size: 0.008271 	 time/step: 0.0302

accept prob:  [1.         1.         0.98997587 1.        ]


step:   2560/5000   accept prob (avg):  0.9979   step size: 0.008271 	 time/step: 0.02048

accept prob:  [0.9942831  0.99725753 1.         1.        ]


step:   2570/5000   accept prob (avg):   0.996   step size: 0.008271 	 time/step: 0.0255

step:   3070/5000   accept prob (avg):  0.9966   step size: 0.008271 	 time/step: 0.01785

accept prob:  [0.9995917  0.99431413 1.         0.99257   ]


step:   3080/5000   accept prob (avg):  0.9993   step size: 0.008271 	 time/step: 0.01738

accept prob:  [0.9982404  1.         1.         0.99910396]


step:   3090/5000   accept prob (avg):  0.9957   step size: 0.008271 	 time/step: 0.01746

accept prob:  [1.         0.9996527  0.98804975 0.9949862 ]


step:   3100/5000   accept prob (avg):  0.9954   step size: 0.008271 	 time/step: 0.01886

accept prob:  [0.99987286 1.         0.98977154 0.99203   ]


step:   3110/5000   accept prob (avg):  0.9996   step size: 0.008271 	 time/step: 0.01755

accept prob:  [0.9986068  0.99966776 1.         1.        ]


step:   3120/5000   accept prob (avg):   0.998   step size: 0.008271 	 time/step: 0.01806

accept prob:  [0.9941964  0.99765325 1.         1.        ]


step:   3130/5000   accept prob (avg):   0.998   step size: 0.008271 	 time/step: 

step:   3620/5000   accept prob (avg):  0.9982   step size: 0.008271 	 time/step: 0.0204

accept prob:  [1.         0.99873954 0.99468744 0.9993425 ]


step:   3630/5000   accept prob (avg):  0.9987   step size: 0.008271 	 time/step: 0.01852

accept prob:  [1.         0.9966169  1.         0.99813205]


step:   3640/5000   accept prob (avg):  0.9974   step size: 0.008271 	 time/step: 0.0211

accept prob:  [0.99598205 0.9942428  0.9992614  1.        ]


step:   3650/5000   accept prob (avg):  0.9982   step size: 0.008271 	 time/step: 0.0201

accept prob:  [1.        0.9959489 1.        0.996945 ]


step:   3660/5000   accept prob (avg):  0.9978   step size: 0.008271 	 time/step: 0.02109

accept prob:  [0.9920195 1.        0.9990592 1.       ]


step:   3670/5000   accept prob (avg):  0.9998   step size: 0.008271 	 time/step: 0.02227

accept prob:  [0.9995641 0.9995248 1.        1.       ]


step:   3680/5000   accept prob (avg):  0.9962   step size: 0.008271 	 time/step: 0.01872

accept

step:   4170/5000   accept prob (avg):  0.9984   step size: 0.008271 	 time/step: 0.02738

accept prob:  [1.         0.99586844 0.99753654 1.        ]


step:   4180/5000   accept prob (avg):  0.9947   step size: 0.008271 	 time/step: 0.02424

accept prob:  [0.99350643 0.99468577 1.         0.9906203 ]


step:   4190/5000   accept prob (avg):  0.9969   step size: 0.008271 	 time/step: 0.02327

accept prob:  [1.        0.994073  0.9934373 1.       ]


step:   4200/5000   accept prob (avg):  0.9984   step size: 0.008271 	 time/step: 0.02275

accept prob:  [1.         0.997592   0.99784964 0.9982704 ]


step:   4210/5000   accept prob (avg):  0.9989   step size: 0.008271 	 time/step: 0.01814

accept prob:  [0.9965405  1.         1.         0.99918586]


step:   4220/5000   accept prob (avg):  0.9978   step size: 0.008271 	 time/step: 0.02303

accept prob:  [1.         0.9997747  1.         0.99130917]


step:   4230/5000   accept prob (avg):  0.9974   step size: 0.008271 	 time/step: 0.02

step:   4720/5000   accept prob (avg):  0.9971   step size: 0.008271 	 time/step: 0.02882

accept prob:  [1.         0.998513   0.99360555 0.99620783]


step:   4730/5000   accept prob (avg):  0.9986   step size: 0.008271 	 time/step: 0.02264

accept prob:  [0.9959464  0.99855566 1.         1.        ]


step:   4740/5000   accept prob (avg):  0.9985   step size: 0.008271 	 time/step: 0.01726

accept prob:  [0.9971733  1.         0.99696743 1.        ]


step:   4750/5000   accept prob (avg):   0.997   step size: 0.008271 	 time/step: 0.01904

accept prob:  [1.        0.9974508 0.9955231 0.995024 ]


step:   4760/5000   accept prob (avg):  0.9959   step size: 0.008271 	 time/step: 0.02001

accept prob:  [0.9948636 0.9934267 1.        0.995203 ]


step:   4770/5000   accept prob (avg):  0.9998   step size: 0.008271 	 time/step: 0.01837

accept prob:  [1.         1.         0.99939835 1.        ]


step:   4780/5000   accept prob (avg):  0.9977   step size: 0.008271 	 time/step: 0.01854


In [39]:
import pickle

with open(model.files['parameters_pkl_file'], 'wb') as f:
    pickle.dump(model.params, f)

In [ ]:
model.sess.graph.collections

In [ ]:
model.sess.graph.get_collection

In [ ]:
model.dynamics.position_fn.summary()

In [ ]:
# Iterate over samples history and calculate observables for each sample.
# `lattice.calc_plaq_observables(samples)` calculates observables for each of
# the samples in the mini-batch.
actions_history = []
avg_plaquettes_history = []
top_charges_history = []
for idx, samples in enumerate(samples_history):
    t0 = time.time()
    observables = np.array(model.lattice.calc_plaq_observables(samples))
    actions, plaqs, charges = observables
    
    actions_history.append(actions)
    avg_plaquettes_history.append(plaqs)
    top_charges_history.append(charges)
    
    print(f'step: {idx}  '
          f'time / step: {time.time() - t0:^6.4g}  '
          f'avg action: {np.mean(actions):^6.4g}  '
          f'avg plaquette: {np.mean(plaqs):^6.4g} '
          f'top charge: {np.mean(charges):^6.4g}')

In [ ]:
_ = helpers.plot_run_data(model.data, 
                          model.params, 
                          model.steps_arr, 
                          model.figs_dir, 
                          skip_steps=1)

In [ ]:
#tf.reset_default_graph()
#model = GaugeModel(params=params,
#                   config=None,
#                   sess=None,
#                   conv_net=False,
#                   hmc=False,
#                   log_dir='../../gauge_logs_graph/run_25',
#                   restore=True)

In [ ]:
samples = np.random.randn(*model.samples.shape)
samples_history = []

In [ ]:
for i in range(500):
    t0 = time.time()
    samples = model.sess.run(model.x_out, feed_dict={model.x: samples})
    samples_history.append(samples)
    print(f'step: {i:^6.4g} time/step: {time.time() - t0:^6.4g}')

In [ ]:
samples_history_conv = np.array(samples_history_conv)
print(samples_history_conv.shape)

In [ ]:
import pickle
samples_history_file = os.path.join(model.info_dir, 'samples_history.pkl')
with open(samples_history_file, 'wb') as f:
    pickle.dump(samples_history_conv, f)